In [1]:
from datetime import date, timedelta

import pandas as pd
import numpy as np
import sys

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
)


In [2]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370464 entries, 0 to 3370463
Data columns (total 5 columns):
id             int64
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
onpromotion    bool
dtypes: bool(1), datetime64[ns](1), int64(3)
memory usage: 106.1 MB


In [3]:
test_m = pd.read_csv('../input/test_m.csv')
test_p = pd.read_csv('../input/test_p.csv')

test_n = df_test[~df_test.id.isin(test_m.id)]
test_n = test_n[~test_n.id.isin(test_p.id)]

In [5]:
test_n.shape

(711056, 5)

In [6]:

df_train = pd.read_csv(
    '../input/train.csv', usecols=[0, 1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    parse_dates=["date"],
    skiprows=range(1, 86672217) #Skip dates before 2016-08-01    
)




In [73]:
df_train_mindate = df_train[['item_nbr','store_nbr','date']].groupby(['item_nbr','store_nbr'])\
    ['date'].min().to_frame('min_date')

#### Find new 
df_train_n = df_train_mindate.loc[ (df_train_mindate['min_date'] >= '2017-08-07'),]

In [74]:
df_train_n.reset_index(inplace=True)

In [91]:
df_train_n.shape

(447, 3)

In [76]:
df_train_sales = pd.merge(df_train, df_train_n,  on=['item_nbr', 'store_nbr'], how='inner' )

In [77]:
df_train_sales.date.min()

Timestamp('2017-08-07 00:00:00')

In [79]:
df_train_sales.shape

(1178, 7)

In [80]:
df_train_n_cntstore = df_train_sales.groupby(['item_nbr'])['store_nbr'].count().to_frame('cnt_store')

In [81]:
df_train_n_sumsales = df_train_sales.groupby(['item_nbr'])['unit_sales'].sum().to_frame('sum_sales')

In [82]:
df_train_n_sumsales.shape

(149, 1)

In [83]:
df_train_n_sumsales.reset_index(inplace=True)
df_train_n_cntstore.reset_index(inplace=True)

train_n_avgsales = pd.merge(df_train_n_sumsales,df_train_n_cntstore,how = 'inner',on=['item_nbr'])

In [92]:
train_n_avgsales['unit_sales'] =train_n_avgsales.sum_sales / train_n_avgsales.cnt_store / 9


In [93]:
train_n_avgsales[['item_nbr', 'unit_sales']].shape

(149, 2)

In [94]:
new_fcst_l15d = pd.merge(test_n, train_n_avgsales, on = ['item_nbr'], how = 'inner')[['id', 'unit_sales']]

In [95]:
new_fcst_l15d.shape

(46144, 2)

In [96]:
new_fcst_l15d.to_csv('../input/new_fcst_l9d.csv',float_format='%.4f', index=None )